[View in Colaboratory](https://colab.research.google.com/github/mayojich/TensorFlow/blob/master/03.%20TensorFlow%20Mechanics%20101.ipynb)

[TensorFlow Mechanics 101](https://www.tensorflow.org/versions/r1.2/get_started/mnist/mechanics)

**TUTORIAL FILES**

In [0]:
# start  training
python fully_connected_feed.py

**Prepare the Data**

**Download**

In [0]:
# input_data.read_data_sets() ensures correct data has been downloaded
# fake_data for unit-testing only
data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

**Input and Placeholders**

In [0]:
# determines shape of inputs
images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                       mnist.IMAGE_PIXELS))
labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))

**Build the Graph**

**Inference**  
Builds graph

In [0]:
# layers are created under unique tf.name_scope
with tf.name_scope('hidden1'):

In [0]:
# layers will use W and b and are generated into tf.Variable
weights = tf.Variable(
    tf.truncated_normal([IMAGE_PIXELS, hidden1_units],
                        stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
    name='weights')
biases = tf.Variable(tf.zeros([hidden1_units]),
                     name='biases')

In [0]:
hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)

In [0]:
hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

In [0]:
logits = tf.matmul(hidden2, weights) + biases

**Loss**

In [0]:
# labels_placeholder values are converted to 64-bit integers
# tf.nn.sparse_..._logits produce 1-hot labels from label_placeholder and compare outputs from inference()
labels = tf.to_int64(labels)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=labels, logits=logits, name='xentropy')

In [0]:
# tf.reduce_mean averages cross entropy values across batch dimensions as total loss
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

**Training**  
Gradient Descent ops minimize loss

In [0]:
# loss tensor is added to tf.summary.scalar for snapshot values of loss
tf.summary.scalar('loss', loss)

In [0]:
# apply gradients with requested learning rate
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [0]:
# Variable to contain counter for global training step
# train_op runs as one full training step
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss, global_step=global_step)

**TRAIN THE MODEL**  
The graph is now built. Train and evaluate the graph with fully_connected_feed.py

**The Graph**

In [0]:
# 'with' indicates all built ops are associated with the default global tf.Graph instance
# tf.Graph is a collection of ops executed as group. Most tf uses the single default graph.
with tf.Graph().as_default():

**The Session**  
All build preparations are created. All operations are generated.

In [0]:
# run graph
sess = tf.Session()

In [0]:
# Session is now a block for scoping
# () = codes will attach to default local session
with tf.Session() as sess:

In [0]:
# to be read as tf.Session.run
init = tf.global_variables_initializer()
sess.run(init)

**Train Loop**

In [0]:
# training loop
for step in xrange(FLAGS.max_steps):
    sess.run(train_op)

**Feed the Graph**

In [0]:
# fill_feed_dict() queries DataSet to be fed to batch_size. Tensors are filled with next images and labels.
images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size,
                                               FLAGS.fake_data)

In [0]:
# python dictionary
# keys = placeholders; values = representative feed tensors.
feed_dict = {
    images_placeholder: images_feed,
    labels_placeholder: labels_feed,
}

**Check the Status**

In [0]:
# train_op and loss are fetched to run.
for step in xrange(FLAGS.max_steps):
    feed_dict = fill_feed_dict(data_sets.train,
                               images_placeholder,
                               labels_placeholder)
    _, loss_value = sess.run([train_op, loss],
                             feed_dict=feed_dict)

In [0]:
# print status every 100 steps for user to know.
if step % 100 == 0:
    print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))

**Visualize the Status**

In [0]:
# all summaries collected into a single Tensor during graph building
summary = tf.summary.merge_all()

In [0]:
# write events files. Contains graph and values of summaries.
summary_writer = tf.summary.FileWriter(FLAGS.train_dir, sess.graph)

In [0]:
# summary values are updated
summary_str = sess.run(summary, feed_dict=feed_dict)
summary_writer.add_summary(summary_str, step)

**Save a Checkpoint**

In [0]:
saver = tf.train.Saver()

In [0]:
saver.save(sess, FLAGS.train_dir, global_step=step)

In [0]:
saver.restore(sess, FLAGS.train_dir)

**EVALUATE THE MODEL**

In [0]:
print('Training Data Eval:')
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.train)
print('Validation Data Eval:')
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.validation)
print('Test Data Eval:')
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.test)

**Build the Eval Graph**

In [0]:
eval_correct = mnist.evaluation(logits, labels_placeholder)

In [0]:
eval_correct = tf.nn.in_top_k(logits, labels, 1)

**Eval Output**

In [0]:
for step in xrange(steps_per_epoch):
    feed_dict = fill_feed_dict(data_set,
                               images_placeholder,
                               labels_placeholder)
    true_count += sess.run(eval_correct, feed_dict=feed_dict)

In [0]:
precision = true_count / num_examples
print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
      (num_examples, true_count, precision))